In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [4]:
from tensorflow.keras.datasets import mnist

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [6]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [7]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [8]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [9]:
model = Sequential()

In [10]:
model.add(Conv2D(16, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [11]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [12]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [14]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [16]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [17]:
model.add(Flatten())

In [18]:
model.add(Dense(54, activation='relu'))

In [19]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 64)          0

#### 組裝

In [21]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [22]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Epoch 1/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0899 - accuracy: 0.1094
Epoch 2/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0897 - accuracy: 0.2423
Epoch 3/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0894 - accuracy: 0.2752
Epoch 4/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0889 - accuracy: 0.2754
Epoch 5/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0881 - accuracy: 0.2851
Epoch 6/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0851 - accuracy: 0.2968
Epoch 7/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0722 - accuracy: 0.4299
Epoch 8/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0498 - accuracy: 0.6631
Epoch 9/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0299 - accuracy: 0.8135
Epoch 10/12
469/469 [==============================] - 1s 3ms/step - loss: 0.0198 - accuracy: 0.8769

### Step 3. 預測

In [23]:
result = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### 看看測試資料表現如何

In [24]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0110 - accuracy: 0.9311


In [25]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 93.11%


In [ ]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [ ]:
from ipywidgets import interact_manual

In [ ]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

<function __main__.my_predict(n)>

### 把我們的 model 存起來

In [ ]:
model.save('myCNNmodel.h5')